In [6]:
import pickle
import tweepy

from BasicAPI import BasicAPI


def lambda_handler(event, context):
    twitter = TwitterAPI(event['lat'], event['lon'])
    return twitter.get_latest_twitter()


class TwitterAPI(BasicAPI):

    type_name = 'twitter'
    twitter_classifier = pickle.load(open('data/CLASSIFIER.obj', 'rb'))

    def __init__(self, user_lat, user_lon, search_distance=BasicAPI.proximity_threshold):
        super(TwitterAPI, self).__init__(user_lat, user_lon, search_distance)
        self.latest_twitter = []

    def get_latest_twitter(self):
        ACCESS_TOKEN = BasicAPI.api_key['Twitter']['ACCESS_TOKEN']
        ACCESS_SECRET = BasicAPI.api_key['Twitter']['ACCESS_SECRET']
        CONSUMER_KEY = BasicAPI.api_key['Twitter']['CONSUMER_KEY']
        CONSUMER_SECRET = BasicAPI.api_key['Twitter']['CONSUMER_SECRET']
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
        api = tweepy.API(auth)
        tweet = []
        for i in range(1):  # Only get nearest 200 * 20 per page
            rst = api.user_timeline('307781209', tweet_mode='extended', page=i)
            tweet += rst
        result = []
        for t in tweet:
            dct = {}
            dct['text'] = t.full_text
            dct['time'] = t.created_at
            result.append(dct)
        self.latest_twitter = result
        self.get_twitter_tag()
        return {'tweets': {'search_distance': self.search_distance,
                           'results': self.latest_twitter
                           }
                }

    def get_twitter_tag(self):
        for t in self.latest_twitter:
            t['tag'] = 'neg'
            if '[' in t['text']:
                t['tag'] = 'pos'
        for t in self.latest_twitter:
            if '[' not in t['text']:
                t['tag'] = TwitterAPI.twitter_classifier.classify(t['text'])


In [7]:
a = TwitterAPI(1.29464, 103.783)

In [8]:
a.get_latest_twitter()

{'tweets': {'results': [{'tag': 'pos',
    'text': '[EWL]: Train svcs btwn #JooKoon &amp; #TuasLink will continue to be suspended till Sun, 19 Nov. Free regular &amp; bridging bus svcs will be avail btwn #JooKoon &amp; #TuasLink all day. Plan your journey &amp; cater for additional train travel time. Alternative transport: https://t.co/4Qu9aPQESU',
    'time': datetime.datetime(2017, 11, 17, 10, 21, 6)},
   {'tag': 'pos',
    'text': '[EWL]: Train svcs btwn #JooKoon &amp; #TuasLink will continue to be suspended tmr. Free regular &amp; bridging bus svcs will be avail btwn #JooKoon &amp; #TuasLink all day tmr. Plan your journey early &amp; cater for additional train travel time on EWL. Other transport: https://t.co/LmN15nP6k6.',
    'time': datetime.datetime(2017, 11, 16, 12, 42, 45)},
   {'tag': 'pos',
    'text': '[NSL]: Train service on the NSL is running normally.',
    'time': datetime.datetime(2017, 11, 15, 20, 44, 1)},
   {'tag': 'pos',
    'text': '[EWL]:Train svc btwn #JooKoon &

In [93]:
BasicAPI.api_key['Twitter']

[{'ACCESS_TOKEN': '2675454162-YfUGh6ff2y1qmDTXPrjlJCT2QckIQ7O8X9WeBLe'},
 {'ACCESS_SECRET': '2Aq398sbkCXKXkLZWBNoWSGkIXmsZP0yQquEXxh6nm2HL'},
 {'CONSUMER_KEY': 'HAHth0DpaQpdHk15i9Xx6l3f8'},
 {'CONSUMER_SECRET': 'xXivCvBS6DsvNg38aAIWQRye2XCV7dLOzWJKXGR07wXotHKP8U'}]

In [85]:
dct = {"Public Taxi/Bus": "8LOiGaQeTXO97oC7KkSYHA==",
       "Ofo": "3e9133f0-a27b-11e7-882a-5937e108b29a",
       "Twitter":{"ACCESS_TOKEN": "2675454162-YfUGh6ff2y1qmDTXPrjlJCT2QckIQ7O8X9WeBLe",
                  "ACCESS_SECRET" : "2Aq398sbkCXKXkLZWBNoWSGkIXmsZP0yQquEXxh6nm2HL",
                  "CONSUMER_KEY" : "HAHth0DpaQpdHk15i9Xx6l3f8",
                  "CONSUMER_SECRET" : "xXivCvBS6DsvNg38aAIWQRye2XCV7dLOzWJKXGR07wXotHKP8U"}
}

In [48]:
import json
with open('API_KEY.json', 'w') as outfile:
    json.dump(dct, outfile)

In [53]:
api_key = {key: value for sub_dict in json.load(open('data/API_KEY.json', 'r')) for key, value in sub_dict.items()}

AttributeError: 'str' object has no attribute 'items'

In [89]:
a = json.load(open('data/API_KEY.json', 'r'))

In [90]:
a['Twitter']

{'ACCESS_SECRET': '2Aq398sbkCXKXkLZWBNoWSGkIXmsZP0yQquEXxh6nm2HL',
 'ACCESS_TOKEN': '2675454162-YfUGh6ff2y1qmDTXPrjlJCT2QckIQ7O8X9WeBLe',
 'CONSUMER_KEY': 'HAHth0DpaQpdHk15i9Xx6l3f8',
 'CONSUMER_SECRET': 'xXivCvBS6DsvNg38aAIWQRye2XCV7dLOzWJKXGR07wXotHKP8U'}

In [18]:
# Import the necessary package to process data in JSON format
import json

# Import the necessary methods from "twitter" library
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

# Variables that contains the user credentials to access Twitter API 
ACCESS_TOKEN = '2675454162-YfUGh6ff2y1qmDTXPrjlJCT2QckIQ7O8X9WeBLe'
ACCESS_SECRET = '2Aq398sbkCXKXkLZWBNoWSGkIXmsZP0yQquEXxh6nm2HL'
CONSUMER_KEY = 'HAHth0DpaQpdHk15i9Xx6l3f8'
CONSUMER_SECRET = 'xXivCvBS6DsvNg38aAIWQRye2XCV7dLOzWJKXGR07wXotHKP8U'

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)

# Get a sample of the public data following through Twitter
iterator = twitter_stream.statuses.sample()

# Print each tweet in the stream to the screen 
# Here we set it to stop after getting 1000 tweets. 
# You don't have to set it to stop, but can continue running 
# the Twitter API to collect data for days or even longer. 
tweet_count = 10
for tweet in iterator:
    tweet_count -= 1
    # Twitter Python Tool wraps the data returned by Twitter 
    # as a TwitterDictResponse object.
    # We convert it back to the JSON format to print/score
    print (json.dumps(tweet))
    
    # The command below will do pretty printing for JSON data, try it out
    # print json.dumps(tweet, indent=4)
       
    if tweet_count <= 0:
        break 

{"in_reply_to_status_id": null, "truncated": false, "geo": null, "coordinates": null, "place": null, "created_at": "Sat Nov 18 11:10:33 +0000 2017", "user": {"time_zone": "Bangkok", "profile_use_background_image": false, "translator_type": "none", "default_profile_image": false, "url": null, "created_at": "Sat May 06 15:39:29 +0000 2017", "profile_image_url_https": "https://pbs.twimg.com/profile_images/926358798054309888/Qvdham3m_normal.jpg", "profile_image_url": "http://pbs.twimg.com/profile_images/926358798054309888/Qvdham3m_normal.jpg", "screen_name": "nnparich", "notifications": null, "contributors_enabled": false, "lang": "th", "id": 860881713110458368, "friends_count": 360, "description": "ASTRO\u2721 | Wanna be your star! | WANNA ONE | \u0e40\u0e21\u0e19\u0e0b\u0e32\u0e19\u0e2e\u0e32\u0e40\u0e2b\u0e19\u0e37\u0e2d\u0e40\u0e21\u0e19\u0e04\u0e37\u0e2d\u0e1e\u0e35\u0e48\u0e21\u0e22\u0e2d\u0e07\u0e08\u0e38\u0e19\u0e40\u0e2b\u0e19\u0e37\u0e2d\u0e1e\u0e35\u0e48\u0e21\u0e22\u0e2d\u0e07\

{"delete": {"timestamp_ms": "1511003434331", "status": {"id_str": "778273248274448384", "user_id_str": "2723478078", "id": 778273248274448384, "user_id": 2723478078}}}
{"delete": {"timestamp_ms": "1511003434411", "status": {"id_str": "931823399998930944", "user_id_str": "1709538842", "id": 931823399998930944, "user_id": 1709538842}}}
{"in_reply_to_status_id": null, "truncated": false, "geo": null, "coordinates": null, "place": null, "created_at": "Sat Nov 18 11:10:33 +0000 2017", "user": {"time_zone": null, "profile_use_background_image": true, "translator_type": "none", "default_profile_image": false, "url": null, "created_at": "Wed Nov 11 23:22:57 +0000 2015", "profile_image_url_https": "https://pbs.twimg.com/profile_images/925201175888109570/nx4aDRE-_normal.jpg", "profile_image_url": "http://pbs.twimg.com/profile_images/925201175888109570/nx4aDRE-_normal.jpg", "screen_name": "camera_ape50", "notifications": null, "contributors_enabled": false, "lang": "ja", "id": 4206302294, "friend

In [51]:
import requests
ACCESS_TOKEN = '2675454162-YfUGh6ff2y1qmDTXPrjlJCT2QckIQ7O8X9WeBLe'
ACCESS_SECRET = '2Aq398sbkCXKXkLZWBNoWSGkIXmsZP0yQquEXxh6nm2HL'
CONSUMER_KEY = 'HAHth0DpaQpdHk15i9Xx6l3f8'
CONSUMER_SECRET = 'xXivCvBS6DsvNg38aAIWQRye2XCV7dLOzWJKXGR07wXotHKP8U'

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
# auth = {'ACCESS_TOKEN' : '2675454162-YfUGh6ff2y1qmDTXPrjlJCT2QckIQ7O8X9WeBLe',
#         'ACCESS_SECRET' : '2Aq398sbkCXKXkLZWBNoWSGkIXmsZP0yQquEXxh6nm2HL',
#         'CONSUMER_KEY' : 'HAHth0DpaQpdHk15i9Xx6l3f8',
#         'CONSUMER_SECRET' : 'xXivCvBS6DsvNg38aAIWQRye2XCV7dLOzWJKXGR07wXotHKP8U'}
# GET statuses/user_timeline
# oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

In [87]:
import requests
from requests_oauthlib import OAuth1
url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
para = {'user_id': '307781209',
        'tweet_mode':'extended'}
result = requests.get(url, auth=auth, params=para)

In [91]:
result.json()[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Nov 17 10:21:06 +0000 2017',
 'display_text_range': [0, 296],
 'entities': {'hashtags': [{'indices': [23, 31], 'text': 'JooKoon'},
   {'indices': [38, 47], 'text': 'TuasLink'},
   {'indices': [152, 160], 'text': 'JooKoon'},
   {'indices': [167, 176], 'text': 'TuasLink'}],
  'symbols': [],
  'urls': [{'display_url': 'bit.ly/2AMuuNK',
    'expanded_url': 'http://bit.ly/2AMuuNK',
    'indices': [273, 296],
    'url': 'https://t.co/4Qu9aPQESU'}],
  'user_mentions': []},
 'favorite_count': 40,
 'favorited': False,
 'full_text': '[EWL]: Train svcs btwn #JooKoon &amp; #TuasLink will continue to be suspended till Sun, 19 Nov. Free regular &amp; bridging bus svcs will be avail btwn #JooKoon &amp; #TuasLink all day. Plan your journey &amp; cater for additional train travel time. Alternative transport: https://t.co/4Qu9aPQESU',
 'geo': None,
 'id': 931467221888524288,
 'id_str': '931467221888524288',
 'in_reply_to_screen_name': None

In [97]:
data = []
for r in result.json():
    dct = {}
    dct['created_at'] = r['created_at']
    dct['full_text'] = r['full_text']
    data.append(dct)

In [98]:
import pandas as pd
df = pd.DataFrame(data)

In [99]:
df

,created_at,full_text
0,Fri Nov 17 10:21:06 +0000 2017,[EWL]: Train svcs btwn #JooKoon &amp; #TuasLin...
1,Thu Nov 16 12:42:45 +0000 2017,[EWL]: Train svcs btwn #JooKoon &amp; #TuasLin...
2,Wed Nov 15 20:44:01 +0000 2017,[NSL]: Train service on the NSL is running nor...
3,Wed Nov 15 20:43:20 +0000 2017,[EWL]:Train svc btwn #JooKoon &amp; #TuasLink ...
4,Wed Nov 15 14:18:03 +0000 2017,[NSL] update: Train service on North South Lin...
5,Wed Nov 15 12:33:26 +0000 2017,[NSL] UPDATE: Due to fewer trains serving the ...
6,Wed Nov 15 12:29:27 +0000 2017,[EWL]UPDATE:Train svcs btwn #JooKoon &amp; #Tu...
7,Wed Nov 15 12:19:43 +0000 2017,[NSL] UPDATE: Free regular bus and bridging bu...
8,Wed Nov 15 10:29:29 +0000 2017,[NSL] UPDATE: Pls add 40 mins train travel tim...
9,Wed Nov 15 09:45:02 +0000 2017,[NSL] UPDATE: Pls add 25 mins train travel tim...


In [110]:
import requests
import pickle
from requests_oauthlib import OAuth1
from BasicAPI import BasicAPI


def lambda_handler(event, context):
    twitter = TwitterAPI(event['lat'], event['lon'])
    return twitter.get_latest_twitter()


class TwitterAPI(BasicAPI):

    type_name = 'twitter'
    # Dependency Issue in AWS
    twitter_classifier = pickle.load(open('data/CLASSIFIER.obj', 'rb'))

    def __init__(self, user_lat, user_lon, search_distance=BasicAPI.proximity_threshold):
        super(TwitterAPI, self).__init__(user_lat, user_lon, search_distance)
        self.latest_twitter = []

    def get_latest_twitter(self):  # Revise without tweepy
        ACCESS_TOKEN = BasicAPI.api_key['Twitter']['ACCESS_TOKEN']
        ACCESS_SECRET = BasicAPI.api_key['Twitter']['ACCESS_SECRET']
        CONSUMER_KEY = BasicAPI.api_key['Twitter']['CONSUMER_KEY']
        CONSUMER_SECRET = BasicAPI.api_key['Twitter']['CONSUMER_SECRET']
        url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
        auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
        para = {'user_id': '307781209',
                'tweet_mode': 'extended'}
        result = requests.get(url, auth=auth, params=para)
        data = []
        for r in result.json():
            dct = {}
            dct['created_at'] = r['created_at']
            dct['full_text'] = r['full_text']
            data.append(dct)
        self.latest_twitter = data
        self.get_twitter_tag()
        return {'tweets': {'search_distance': self.search_distance,
                           'results': self.latest_twitter
                           }
                }

    def get_twitter_tag(self):
        for t in self.latest_twitter:
            t['tag'] = 'neg'
            if '[' in t['full_text']:
                t['tag'] = 'pos'
        for t in self.latest_twitter:
            if '[' not in t['full_text']:
                t['tag'] = TwitterAPI.twitter_classifier.classify(t['full_text'])


In [111]:
a = TwitterAPI(1.29464, 103.783)

In [112]:
a.get_latest_twitter()

{'tweets': {'results': [{'created_at': 'Fri Nov 17 10:21:06 +0000 2017',
    'full_text': '[EWL]: Train svcs btwn #JooKoon &amp; #TuasLink will continue to be suspended till Sun, 19 Nov. Free regular &amp; bridging bus svcs will be avail btwn #JooKoon &amp; #TuasLink all day. Plan your journey &amp; cater for additional train travel time. Alternative transport: https://t.co/4Qu9aPQESU',
    'tag': 'pos'},
   {'created_at': 'Thu Nov 16 12:42:45 +0000 2017',
    'full_text': '[EWL]: Train svcs btwn #JooKoon &amp; #TuasLink will continue to be suspended tmr. Free regular &amp; bridging bus svcs will be avail btwn #JooKoon &amp; #TuasLink all day tmr. Plan your journey early &amp; cater for additional train travel time on EWL. Other transport: https://t.co/LmN15nP6k6.',
    'tag': 'pos'},
   {'created_at': 'Wed Nov 15 20:44:01 +0000 2017',
    'full_text': '[NSL]: Train service on the NSL is running normally.',
    'tag': 'pos'},
   {'created_at': 'Wed Nov 15 20:43:20 +0000 2017',
    'ful